In [1]:
import matplotlib.pyplot as plt
import numpyro
import numpyro.distributions as dist
import numpyro_stein.stein.kernels as kernels
from numpyro_stein.stein.autoguides import AutoDelta
from numpyro_stein.util import init_with_noise
from numpyro_stein.stein.stein import SVGD
from numpyro_stein.distributions.normal_mixture_distribution import NormalMixture
from numpyro_stein.stein.kernels import SteinKernel
from numpyro.infer import ELBO
from numpyro.infer.util import init_to_value
import seaborn as sns
import os
import jax
import jax.numpy as np
from random import randint

NameError: name 'SteinKernel' is not defined

In [ ]:
rng_key = jax.random.PRNGKey(randint(0, int(1e6)))
num_iterations = 1500

In [ ]:
def model():
    numpyro.sample('x', NormalMixture(np.array([1/3, 2/3]), 
                                      np.array([-2., 2.]), np.array([1., 1.])))

In [ ]:
guide = AutoDelta(model, init_strategy=init_with_noise(init_to_value({'x': -10.}), noise_scale=1.0))
svgd = SVGD(model, guide, numpyro.optim.Adagrad(step_size=1.0), ELBO(),
            kernels.RBFKernel(bandwidth_factor=lambda n: 1.), num_stein_particles=100, num_loss_particles=3)
svgd_state = svgd.init(rng_key)

In [ ]:
sns.kdeplot(svgd.get_params(svgd_state)['auto_x'])


In [ ]:
for i in range(num_iterations):
    svgd_state, loss = svgd.update(svgd_state)
    if i % 100 == 0:
        print(loss)

In [ ]:
plt.clf()
sns.kdeplot(svgd.get_params(svgd_state)['auto_x'])

In [ ]:
svgd.get_params(svgd_state)